# Can Be used for other Datasets.

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from proto_dist_ml.mglvq import MGLVQ

In [3]:
iris = load_iris()

In [4]:
input_data = load_iris().data
data_label = load_iris().target
prototype_per_class = 3
epochs = 2

In [5]:
X_train, X_test, y_train, y_test = train_test_split(input_data,
                                                    data_label,
                                                    test_size=0.3,
                                                    random_state=42)

In [6]:
clf = MGLVQ(K = prototype_per_class, T = epochs)

In [10]:
# levenshtein distance as it is non-eucledian distance measure.
def levenshtein(x, y):
    m = len(x)
    n = len(y)
    Delta = np.zeros((m+1, n+1), dtype=int)
    for i in range(m):
        Delta[i+1, 0] = Delta[i, 0] + 1
    for j in range(n):
        Delta[0, j+1] = Delta[0, j] + 1
    for i in range(m):
        for j in range(n):
            delta_ij = 0 if x[i] == y[j] else 1
            Delta[i+1, j+1] = np.min([delta_ij + Delta[i,j], 1 + Delta[i+1, j], 1 + Delta[i, j+1]])
    return Delta[m][n]

# Levenshtein distance between elements
X_train_new = np.zeros((len(X_train), len(X_train)))
for k in range(len(X_train)):
    x = X_train[k]
    for l in range(len(X_train)):
        y = X_train[l]
        X_train_new[k, l] = levenshtein(x, y)
        X_train_new[l, k] = X_train_new[k, l]

In [11]:
mglvq = clf.fit(X_train_new, y_train)

In [12]:
X_train_new_small = X_train_new[:, mglvq._w]
print('MGLVQ classifier has an accuracy of %g%% using test data' % (mglvq.score(X_train_new_small, y_train) * 100))

MGLVQ classifier has an accuracy of 77.1429% using test data
